In [4]:
import pandas as pd
%matplotlib inline
import seaborn as sns
import numpy as np

In [5]:
units = pd.read_csv("../data/510049986/raw_data.csv", encoding="windows-1252", index_col=0)

/var/folders/s7/d_hp0cc12jj5nrlqny507k2h0000gn/T/ipykernel_96029/1778539441.py:1: DtypeWarning: Columns (2,7,8,35,40,68,73,84,98,106,108,127,154,161,162,164,165,166,169,177,179,198,225,232,233,235,236,237,240,248,250,269,296,303,304,306,307,308,311,319,321,340,367,374,375,377,378,379,382,390,392,411,438,445,446,448,449,450,452,461,489,527,543,551,552,554,564,585,618,627,639,660,670,693,700,712,745,751,759,761,766,767,774,782,806,809,816,817,818,819,820,821,824,832,834,840,847,855,882,889,890,891,892,893,894,897,905,907,912,926,953,960,961,963,964,965,968,976,978,984,999,1026,1033,1034,1036,1037,1038,1041,1049,1051,1056,1070,1097,1104,1105,1107,1108,1109) have mixed types. Specify dtype option on import or set low_memory=False.
  units = pd.read_csv("../data/510049986/raw_data.csv", encoding="windows-1252", index_col=0)


In [6]:
units_ = units[~units.apply(lambda srs: 'Deprec!' in srs['NameInMenu'], axis='columns')]

In [7]:
units_[['NameInMenu', 'Weapon1PhysicalDamages', 'Weapon1Arme', 'Weapon2PhysicalDamages', 'Weapon2Arme']]

,NameInMenu,Weapon1PhysicalDamages,Weapon1Arme,Weapon2PhysicalDamages,Weapon2Arme
0,DANA,7.0,3.0,NaN,3.0
1,RZP KUB-M,9.0,3.0,NaN,NaN
2,2K22 TUNGUSKA-M,1.0,3.0,7.0,3.0
3,2K22 TUNGUSKA,1.0,3.0,7.0,3.0
4,2S15 NOROV,1.0,17.0,3.0,3.0
...,...,...,...,...,...
1813,F-16A BLOCK 15,1.0,3.0,3.0,54.0
1814,TORNADO ECR,1.0,54.0,4.0,3.0
1815,TORNADO IDS,1.0,3.0,15.0,3.0
1816,TORNADO MFG,1.0,3.0,4.0,54.0


In [8]:
import re

def weaponify(unit):
    # We're going to build a new pandas Series with all of the information we want.
    srs = pd.Series()
    
    # First thing first, how many weapons are we talking about? A quick regex will get us this fact.
    try:
        top_weapon_number = max([int(re.search(r'\d+', c).group()) for c in unit.index\
                                 if ("Weapon" in c) and (pd.notnull(unit[c]))])
    except ValueError:
        # There are none!
        top_weapon_number = 0
    
    # Some things we need to keep track of outside of the loop.
    salvo_stock_indices_already_visited = []
    
    for i in range(1, top_weapon_number + 1):
#         import pdb; pdb.set_trace()
        
        fragment = 'Weapon{0}'.format(i)
        salvo_index = 0 if unit[fragment + 'SalvoStockIndex'] == "null" else int(unit[fragment + 'SalvoStockIndex'])
        
        # If we haven't already visited this stock index, this weapon is the first one there.
        # Let's copy its data into our new Series!
        if salvo_index not in salvo_stock_indices_already_visited:
            columns_of_interest = [c for c in unit.index if fragment in c]
            for c in columns_of_interest:
                srs[c] = unit[c]
        # If we have already visited this stock index, let's find out why.
        else:
            # Before we begin, let's find that previous weapon.
            # We can find it by taking the index of the previous weapon's position in the visited list and adding one.
            previous_fragment = "Weapon{0}".format(salvo_stock_indices_already_visited.index(salvo_index) + 1)
            
            # Case 1: The new (current) weapon is a smoke weapon. 
            # We see this when Arme == 3 and PhysicalDamages is "null".
            if (unit[fragment + 'Arme'] == "null" or int(unit[fragment + 'Arme']) == 3) \
                and (unit[fragment + 'PhysicalDamages'] == "null"):
                # Attach a can-smoke tag to the weapon.
                srs[previous_fragment + 'CanSmoke'] = True
            # Case 2a: The first weapon is an SMG, and the second weapon is also an SMG.
            # SMGs get doubled accuracy when the fight is in CQC. It turns out that this is implemented within the
            # engine by, no joke, simply attaching another copy of the weapon to the unit which points to the same
            # exact thing except that accuracy is doubled.
            #
            # For an example, refer to the French Groupe Commandat command infantry unit, which is the 15th unit
            # in the list of them on the Modding Suite (at the moment at least).
            #
            # To detect this case, we check to see if both weapons are SMGs.
            #
            # Case 2b: Similarly, if the the weapon is a battle rifle, expect half accuracy in CQC.
            # Case: 2c: If the weapon is an assault rifle, expect the same accuracy. But the weapon gets split anyway!
            elif (unit[fragment + 'TypeArme'] == "D1D5010000000000" \
                    and unit[previous_fragment + 'TypeArme'] == "D1D5010000000000") or \
                (unit[fragment + 'TypeArme'] == "6ABCBA1C00000000" \
                    and unit[previous_fragment + 'TypeArme'] == "6ABCBA1C00000000") or \
                (unit[fragment + 'TypeArme'] == "5C99C7BA89E30B00" \
                    and unit[previous_fragment + 'TypeArme'] == "5C99C7BA89E30B00"):
                    # In this case we do...nothing! The data is already there.
                    pass
            # Case 3: The new (current) weapon is the HE or AP component of the previous weapon.
            # These can occur in either order, so we have to make sense of it by looking at what the values are.
            # Taking a maximum does the job nicely, but first we have to fix nullity.
            else:
                # Then, we play the game.
                prior_arme = srs[previous_fragment + 'Arme']
                if prior_arme == 'null': prior_arme = np.nan
                curr_arme = unit[fragment + 'Arme']
                if curr_arme == 'null': curr_arme = np.nan
                prior_physical = unit[previous_fragment + 'PhysicalDamages']
                if prior_physical == 'null': prior_physical = np.nan
                curr_physical = unit[fragment + 'PhysicalDamages']
                if curr_physical == 'null': curr_physical = np.nan
                # The rathered tortured semantics are necessary to work around numpy bitching about types.
                arme = int(np.nanmax(np.array([prior_arme, curr_arme]).astype(float)))
                physical = int(np.nanmax(np.array([prior_physical, curr_physical]).astype(float)))
                # Attach the new values.
                srs[previous_fragment + 'Arme'] = arme
                srs[previous_fragment + 'PhysicalDamages'] = physical

        # Finally, don't forget to add the stock index to the visited list!
        salvo_stock_indices_already_visited.append(salvo_index)

    # We may have produced empties along the way, populating fields in a pattern like ['Weapon1', 'Weapon3'] instead of
    # e.g. ['Weapon1', 'Weapon2']. This is much easier to fix afterwards, though, then during. So let's fix that now!
    # Start with the same regex as before...
    try:
        top_weapon_number = max([int(re.search(r'\d+', c).group()) for c in srs.index\
                                 if ("Weapon" in c) and (pd.notnull(srs[c]))])
    except:
        return srs
    
    # Assuming we have weapons at all (the duck test above caught the case that we don't), we keep going.
    attached_weapon_numbers = [i for i in range(1, top_weapon_number + 1) if 'Weapon{0}Arme'.format(i) in srs.index]
    
    # Figure out how to map the attached weapon numbers.
    trips = zip(attached_weapon_numbers, range(1, len(attached_weapon_numbers) + 1))
    
    # Map down.
    for origin, destination in trips:
        if origin == destination:
            pass
        else:
            srs.index = [c if (('Weapon' not in c) or (int(re.search(r'\d+', c).group()) != origin))\
                            else c.replace(re.search(r'\d+', c).group(), str(destination)) for c in srs.index]
    
    return srs

In [9]:
units.query('NameInMenu == "RECCE #reco2"').iloc[0][['Weapon3SalvoStockIndex']]

Weapon3SalvoStockIndex    1.0
Name: 207, dtype: object

In [10]:
weaponify(units.query('NameInMenu == "KUB-M"').iloc[0])[['Weapon1Arme', 'Weapon1PhysicalDamages',
                                                          'Weapon2Arme', 'Weapon2PhysicalDamages', 
                                                          'Weapon3Arme', 'Weapon3PhysicalDamages']]  # Single weapon test

/var/folders/s7/d_hp0cc12jj5nrlqny507k2h0000gn/T/ipykernel_96029/2891857363.py:5: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  srs = pd.Series()


ValueError: cannot convert float NaN to integer

In [ ]:
weaponify(units.query('NameInMenu == "AMX-30"').iloc[0])[['Weapon1Arme', 'Weapon1PhysicalDamages',
                                                          'Weapon2Arme', 'Weapon2PhysicalDamages', 
                                                          'Weapon3Arme', 'Weapon3PhysicalDamages']]  # Single AP/HE test

In [ ]:
weaponify(units.query('NameInMenu == "AMX-30B"').iloc[0])[['Weapon1Arme', 'Weapon1PhysicalDamages',
                                                          'Weapon2Arme', 'Weapon2PhysicalDamages', 
                                                          'Weapon3Arme', 'Weapon3PhysicalDamages']]  # Double HE-AP test

In [ ]:
weaponify(units.query('NameInMenu == "DANA"').iloc[0])[['Weapon1Arme', 'Weapon1PhysicalDamages',
                                                          'Weapon2Arme', 'Weapon2PhysicalDamages', 
                                                          'Weapon3Arme', 'Weapon3PhysicalDamages']]  # Smoke test

In [ ]:
weaponify(units_.ix[18])[['Weapon1PhysicalDamages', 'Weapon1Arme', 'Weapon2Arme', 
                          'Weapon3PhysicalDamages', 'Weapon3Arme']]  # Test: Assault Rifle

In [ ]:
weaponify(units_.ix[17])[['Weapon1PhysicalDamages', 'Weapon1Arme', 'Weapon2Arme', 
                          'Weapon3PhysicalDamages', 'Weapon3Arme']]  # Test: Battle Rifle

In [ ]:
weaponify(units.iloc[15])[['Weapon1PhysicalDamages', 'Weapon1Arme', 'Weapon2Arme', 'Weapon2PhysicalDamages',
                          'Weapon3PhysicalDamages', 'Weapon3Arme']]  # Test: SMG

In [ ]:
weaponify(units.query('NameInMenu == "#command UDALOY II"').iloc[0])[
    ['Weapon1Arme', 'Weapon1PhysicalDamages', 'Weapon2Arme', 'Weapon2PhysicalDamages',
     'Weapon3Arme', 'Weapon3PhysicalDamages', 'Weapon4Arme', 'Weapon4PhysicalDamages',
     'Weapon5Arme', 'Weapon5PhysicalDamages', 'Weapon6Arme', 'Weapon6PhysicalDamages',
     'Weapon7Arme', 'Weapon7PhysicalDamages', 'Weapon8Arme', 'Weapon8PhysicalDamages',
     'Weapon9Arme', 'Weapon9PhysicalDamages', 'Weapon10Arme', 'Weapon10PhysicalDamages',
     'Weapon11Arme', 'Weapon11PhysicalDamages',]]
# Test: biggest ship

In [ ]:
weaponify(units.query('NameInMenu == "#command NAJIN"').iloc[0])[
    ['Weapon1Arme', 'Weapon1PhysicalDamages', 'Weapon2Arme', 'Weapon2PhysicalDamages',
     'Weapon3Arme', 'Weapon3PhysicalDamages', 'Weapon4Arme', 'Weapon4PhysicalDamages',
     'Weapon5Arme', 'Weapon5PhysicalDamages', 'Weapon6Arme', 'Weapon6PhysicalDamages',
     'Weapon7Arme', 'Weapon7PhysicalDamages', 'Weapon8Arme', 'Weapon8PhysicalDamages',
     'Weapon9Arme', 'Weapon9PhysicalDamages', 'Weapon10Arme', 'Weapon10PhysicalDamages',
     'Weapon11Arme', 'Weapon11PhysicalDamages']]
# Test: big ship.

In [ ]:
weaponify(units.query('NameInMenu == "AML 60/20 SERVAL"').iloc[0])[
    ['Weapon1PhysicalDamages', 'Weapon1Arme', 'Weapon2Arme', 'Weapon2PhysicalDamages',
     'Weapon3PhysicalDamages', 'Weapon3Arme']]
# Test: complex

In [ ]:
weaponify(units.query('NameInMenu == "AH-64A APACHE"').iloc[0])[
    ['Weapon1PhysicalDamages', 'Weapon1Arme', 'Weapon2Arme', 'Weapon2PhysicalDamages',
     'Weapon3PhysicalDamages', 'Weapon3Arme']]
# Test: complex

In [ ]:
from checkpoints import checkpoints
checkpoints.enable()

units__ = units_.safe_apply(weaponify, axis='columns')

In [ ]:
units__.to_csv("../data/510049986/intermediate_data.csv")